In [1]:
import os 

import fiona #must be import before geopandas
import geopandas as gpd
import rasterio
import rioxarray
import re
import rtree
import shapely
import pickle

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook
import tqdm
# Standard packages
import tempfile
import warnings
import urllib
import shutil

# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

from lxml.etree import Element,SubElement,tostring
import xml.dom.minidom
from xml.dom.minidom import parseString
import xml.etree.ElementTree as et
from xml.dom import minidom

#import requests
from PIL import Image
from io import BytesIO
import tqdm
from skimage.metrics import structural_similarity as compare_ssim
import imutils

import psutil

import data_eng.az_proc as ap
import data_eng.form_calcs as fc

## File Paths

In [2]:
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

tile_names_tile_urls_complete_array = np.load("image_download_azure/tile_name_tile_url_complete_array.npy")

tiles_labeled = "tile_name_tile_url_labeled.npy"
tiles_labeled_from_complete_set = np.load(tiles_labeled)

tracker_file_path = 'outputs/tile_img_annotation_annotator.npy'
tile_img_annotation = np.load(tracker_file_path)
tile_img_annotation_annotator = np.load("outputs/tile_img_annotation_annotator.npy")

tiles_errors = 'tile_name_tile_url_error_downloading.npy'
tiles_errors = np.load(tiles_errors)

#create folder to hold tiles in completed dataset

tiles_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles")
tiles_xml_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles_xml")
os.makedirs(tiles_complete_dataset_path, exist_ok=True)
tiles_xml_list = os.listdir(tiles_xml_complete_dataset_path)
new_tiles_xml_complete_dataset_path = os.path.join(parent_directory,"complete_dataset","tiles_xml_merged")

tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")
image_characteristics = pd.read_csv("image_characteristics.csv")
tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")

#NAIP quad map
quad_indicies_path = "C:/Users/rapiduser/Box/EPA STAR 2019 (Community Resistance to Environmental Disasters)/Data/AST Datasets/MapIndices_National_GDB/MapIndices_National_GDB.gdb"
#fiona.listlayers(quad_indicies_path)
rechipped_image_path = os.path.join(parent_directory,"temp/rechip")

folders_of_images_xmls_by_tile = os.listdir(rechipped_image_path)
images_and_xmls_by_tile_path = rechipped_image_path

In [3]:
tiles_dir = tiles_complete_dataset_path
tiles_xml_path = tiles_xml_complete_dataset_path
item_dim = int(512)

# Add chips to rechip folder

In [ ]:
#for tile_name in tqdm.tqdm(folders_of_images_xmls_by_tile):
#    add_chips_to_chip_folders(rechipped_image_path, tile_name)

# Great Tables of characteristics for tiles/images

In [4]:
#tile_characteristics, image_characteristics = fc.image_tile_characteristics(rechipped_image_path, tiles_complete_dataset_path)
#tile_characteristics.to_csv("tile_characteristics.csv")
#image_characteristics.to_csv("image_characteristics.csv")

100%|██████████| 2139/2139 [6:52:33<00:00, 11.57s/it]  


# Generate tile XMLs

In [ ]:
#make consistent labels 
#for tile_name in tqdm.tqdm(folders_of_images_xmls_by_tile):
    #specify image/xml paths for each tile
#    positive_xml_dir = os.path.join(images_and_xmls_by_tile_path, tile_name, "chips_positive_xml")
#    fc.correct_inconsistent_labels_xml(positive_xml_dir)

In [8]:
fc.generate_tile_xmls(rechipped_image_path, tiles_complete_dataset_path, tiles_xml_complete_dataset_path, int(512))

100%|██████████| 2139/2139 [1:17:33<00:00,  2.18s/it]


## Combine neighbooring bounding boxes

https://answers.opencv.org/question/231263/merging-nearby-rectanglesedited/
https://stackoverflow.com/questions/55593506/merge-the-bounding-boxes-near-by-into-one

In [7]:
fc.merge_tile_annotations(tiles_xml_list,tiles_xml_complete_dataset_path,new_tiles_xml_complete_dataset_path,5)

100%|██████████| 2120/2120 [09:54<00:00,  3.56it/s]


# Clip storm surge data

In [ ]:
tile.coords
clip based on max min
#https://automating-gis-processes.github.io/CSC18/lessons/L6/clipping-raster.html


In [ ]:
def getFeatures(gdf):
    """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
    import json
    return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
bbox = box(minx, miny, maxx, maxy) #specify bounding box 
geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
geo = geo.to_crs(crs=data.crs.data)
coords = getFeatures(geo)
In [20]: out_img, out_transform = mask(raster=data, shapes=coords, crop=True)


In [ ]:
- for each tank
    - get se sw coordinates (by pixel) of each object with respect to tif
        - (The function by image can be adapted:image_characteristics(tiles_dir, verified_positive_jpgs))
    -determine whether the width or height of the object is less.
        -if equal use the width. with(in pixels) *0.6(m) is the diameter.
    - get the lat lon for each se sw coordinates for each tank 
- subset slosh data based on tank lat lon 

In [ ]:
In [13]: minx, miny = 24.60, 60.00

In [14]: maxx, maxy = 25.22, 60.35

In [15]: bbox = box(minx, miny, maxx, maxy)

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2)
ax1.set_title('correct_image')
ax1.imshow(gray_known_image)
ax2.set_title('labeled_chip_array')
ax2.imshow(gray_labeled_image)
plt.show()